# AUTOMATED ESSAY GRADING WITH NEURAL NETWORKS
Essays are crucial testing tools for assessing academic achievement, integration of ideas and ability to recall, but are expensive and time consuming to grade manually. The purpose of this project is to implement and train neural network to automatically assess and grade essays.


Importing all the required libraries

In [573]:
import nltk
import statistics
import numpy as np
import pickle as pkl
import autocorrect
from spellchecker import SpellChecker
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

Setting up the data directory

In [17]:
DATASET_DIR = './data/'

Extracting the Data using pandas and dropping columns that are irrelevant for essay grading

In [21]:
X = pd.read_csv((DATASET_DIR + 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['essay_set','rater1_domain1', 'rater2_domain1'])

Extracting data only from the essay set 1. This is to help in faster training and testing of the model by keeping the dataset small.

In [120]:
X1 = pd.read_csv((DATASET_DIR + 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
X1 = X1.dropna(axis=1)
X1 = X1.drop(columns=['rater1_domain1', 'rater2_domain1'])
X1 = X1[X1['essay_set'] == 1]
y1 = X1['domain1_score']
X1.shape

(1783, 4)

Method to retrieve GloVe word embeddings from the glove embeddings txt file.

In [2]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    model = {}
    for line in f:
        try:
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            print(word)
    print("Done.",len(model)," words loaded!")
    return model

Extracting the Glove Embeddings in a variable

In [3]:
glove_embeddings = loadGloveModel('glove.840B.300d.txt')

Loading Glove Model
.
at
.
to
.
.
email
or
contact
Email
on
At
by
in
emailing
Contact
at
•
at
is
Done. 2195884  words loaded!


Method to create word to vector embeddings of the essay. This method takes in the sentences in each essay and returns an embedding for it.

In [ ]:
def word2vec(sent, stop_words=False):
    w2v = load_w2v()
    w2v_mean = np.mean(list(w2v.values()))
    w2v_std = np.std(list(w2v.values()))
    embed_size = 300

    embeddings = np.random.normal(w2v_mean, w2v_std, (len(sent), embed_size))
    for i in range(0, len(sent)):
        tokens = tokenize_words(sent[i], remove_stop_words=stop_words)[0]
        count = 0
        for token in tokens:
            if token in w2v:
                count += 1
                embeddings[i] += w2v[token]
        embeddings[i] /= count
    return embeddings

Method to get the list of misspelled words, corrected errors and the total number of misspelled words

In [102]:
# returns list of misspelled words, corrected errors and the total number of misspelled words
def typos(words):
    spell = SpellChecker()

    # find those words that may be misspelled
    misspelled = spell.unknown(words)
    corrected_words = {}
    for word in misspelled:
        # Get the one `most likely` answer
        correct_spell = spell.correction(word)
        if correct_spell != word:
            corrected_words[word] = correct_spell

    misspelled = [word for word in corrected_words]
    corrected_words = [corrected_words[word] for word in corrected_words]
    return misspelled, corrected_words, len(misspelled)
def spelling_errors(data):
    words = tokenize_words(data)[0]
    # 0.7 because a lot of words are correct but marked as spelling errors
    return int(typos(words)[2]*0.7)

Method to get the list of stop words

In [57]:
# returns the list of stop words
def get_stop_words():
    stop_words = nltk.corpus.stopwords.words('english')
    return stop_words

Method to separate the sentences and get the total number of sentences in an essay

In [70]:
# returns all sentences and total number of sentences
def tokenize_sentences(data):
    sent_token = nltk.tokenize.sent_tokenize(data)
    return sent_token, len(sent_token)

# calls the tokenize_sentences method and returns the total number of sentences
def sent_count(data):
    return tokenize_sentences(data)[1]

Method to get the tokens in the essay. There are multiple parameters based on which the final output is returned. The method returns all the tokens, types and the total number of tokens

In [68]:
# returns all tokens, all types and total number of tokens
# if punc is False(default True), then punctuations are not removed
# If lower is False(default True), then words are not kept as is, not converted to lower case
def tokenize_words(data, punc=True, remove_stop_words=False, lower=True):
    if lower:
        word_tokens = nltk.tokenize.word_tokenize(data.lower())
    else:
        word_tokens = nltk.tokenize.word_tokenize(data)
    if punc:
        word_tokens = [word for word in word_tokens if word.isalnum()]
    if remove_stop_words:
        stop_words = get_stop_words()
        word_tokens = [word for word in word_tokens if word not in stop_words]
        
    return word_tokens, list(set(word_tokens)), len(word_tokens)

# calls the tokenize_words method and returns the total number of tokens
def word_count(data):
    return tokenize_words(data)[2]

Method to get the frequency of each token when a list of tokens in passed to it.

In [ ]:
# returns frequency of each word
def token_frequency(words):
    frequency_of_tokens = nltk.FreqDist(words)
    return frequency_of_tokens

Method to get length of each word in the essay

In [85]:
# returns length of each word in token
def token_length(data):
    words = tokenize_words(data)[0]
    len_of_tokens = {}
    for word in words:
        len_of_tokens[word] = len(word)
    return len_of_tokens
# calls the token_length method and returns the average of token lengths
def avg_length(words):
    return statistics.mean(token_length(words).values())

Method to get lenght of sentences in number of words and the average number of words in a sentence

In [570]:
# returns length of sentences in terms of number of words and the average length of sentences
def sentence_length(sent):
    len_of_sent = []
    for s in sent:
        len_of_sent.append(tokenize_words(s)[2])
    return len_of_sent, statistics.mean(len_of_sent)
def avg_word_sentence(data):
        return sentence_length(tokenize_sentences(data)[0])[1]

Method that counts the number of characters in an essay

In [51]:
def char_count(data):
    return len(data.lower().replace(' ',''))

Method to get the part of speech tags for each word in the sentence and return the count of nouns, adjectives, verbs and adverbs

In [90]:
def pos_tags(data):
    sent = tokenize_sentences(data)[0]
    
    noun_count = 0
    adj_count = 0
    verb_count = 0
    adv_count = 0
    
    for s in sent:
        tags = nltk.pos_tag(tokenize_words(s)[0])
        for tag in tags:
            if tag[1][0] == 'N':
                noun_count += 1
            elif tag[1][0] == 'J':
                adj_count += 1
            elif tag[1][0] == 'V':
                verb_count += 1
            elif tag[1][0] == 'R':
                adv_count += 1
    return noun_count,adj_count,verb_count,adv_count

For this project, only essay set 1 was used for analysis and model creation.

Extacted Features:
1. Number of characters
2. Number of words
3. Number of sentences
4. Average word length
5. Average number of words in a sentence
6. Noun Count
7. Adjective Count
8. Verb Count
9. Adjective Count 
10. Spelling errors

In [312]:
# 1. Number of characters in an essay
# X['num_chars'] = X['essay'].apply(char_count)
# X.head()
X1['num_chars'] = X1['essay'].apply(char_count)
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,-0.277644,-0.746351,-0.007471,-0.532164,-0.708112,-0.204692,0.065043
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,0.453147,-0.305963,-0.027029,0.394898,-0.263516,0.432873,-0.589108
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,-0.714439,-0.966546,-0.131474,-0.377654,-0.441354,-0.842257,-0.869458
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,1.309936,0.464717,2.160992,2.249023,1.248110,0.911046,0.812644
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,0.822742,0.795009,0.333307,0.456703,0.358918,0.592264,1.092995


In [313]:
# 2. Number of words in an essay
# X['num_words'] = X['essay'].apply(word_count)
# X.head()
X1['num_words'] = X1['essay'].apply(word_count)
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,331,-0.746351,-0.007471,-0.532164,-0.708112,-0.204692,0.065043
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,418,-0.305963,-0.027029,0.394898,-0.263516,0.432873,-0.589108
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,279,-0.966546,-0.131474,-0.377654,-0.441354,-0.842257,-0.869458
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,520,0.464717,2.160992,2.249023,1.248110,0.911046,0.812644
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,462,0.795009,0.333307,0.456703,0.358918,0.592264,1.092995


In [314]:
# 3. Number of sentences in an essay
# X['num_sents'] = X['essay'].apply(sent_count)
# print(X.head())
X1['num_sents'] = X1['essay'].apply(sent_count)
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,331,16,-0.007471,-0.532164,-0.708112,-0.204692,0.065043
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,418,20,-0.027029,0.394898,-0.263516,0.432873,-0.589108
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,279,14,-0.131474,-0.377654,-0.441354,-0.842257,-0.869458
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,520,27,2.160992,2.249023,1.248110,0.911046,0.812644
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,462,30,0.333307,0.456703,0.358918,0.592264,1.092995


In [315]:
# 4. Average word length of an essay
# X['avg_word_length'] = X['essay'].apply(avg_length)
# print(X.head())
X1['avg_word_length'] = X1['essay'].apply(avg_length)
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,331,16,5.006369,-0.532164,-0.708112,-0.204692,0.065043
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,418,20,5.000000,0.394898,-0.263516,0.432873,-0.589108
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,279,14,4.965986,-0.377654,-0.441354,-0.842257,-0.869458
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,520,27,5.712551,2.249023,1.248110,0.911046,0.812644
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,462,30,5.117347,0.456703,0.358918,0.592264,1.092995


In [571]:
# 5. Average number of words in a sentence
# X['avg_word_sent'] = X['essay'].apply(avg_length)
# print(X.head())
X1['avg_word_sent'] = X1['essay'].apply(avg_word_sentence)
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count,spelling_erros,avg_word_sent
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,331,16,5.006369,74,19,69,24,7,20.687500
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,418,20,5.000000,104,24,85,17,12,20.900000
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,279,14,4.965986,79,22,53,14,5,19.928571
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,520,27,5.712551,164,41,97,32,32,19.259259
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,462,30,5.117347,106,31,89,35,7,15.400000


In [569]:
# 10. Number of spelling errors in an essay
# X['spelling_erros'] = X['essay'].apply(spelling_errors)
# X.head()
X1['spelling_erros'] = X1['essay'].apply(spelling_errors)
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count,spelling_erros
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,331,16,5.006369,74,19,69,24,7
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,418,20,5.000000,104,24,85,17,12
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,279,14,4.965986,79,22,53,14,5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,520,27,5.712551,164,41,97,32,32
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,462,30,5.117347,106,31,89,35,7


In [316]:
# 6. Number of nouns in an essay
# 7. Number of adjectives in an essay
# 8. Number of verbs in an essay
# 9. Number of adverbs in an essay
# X['noun_count'], X['adj_count'], X['verb_count'], X['adv_count'] = zip(*X['essay'].map(pos_tags))
# X.head()
X1['noun_count'], X1['adj_count'], X1['verb_count'], X1['adv_count'] = zip(*X1['essay'].map(pos_tags))
X1.head()

,essay_id,essay_set,essay,domain1_score,num_chars,num_words,num_sents,avg_word_length,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",8,1538,331,16,5.006369,74,19,69,24
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,1870,418,20,5.000000,104,24,85,17
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,1263,279,14,4.965986,79,22,53,14
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,2642,520,27,5.712551,164,41,97,32
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,2105,462,30,5.117347,106,31,89,35


Creating the Vocabulary of the essays in set 1

In [ ]:
vocab = []
vocab_size = set()
for i in range(0,len(X1)):
    tokens = tokenize_words(X1.loc[i]['essay'])[1]
    print(tokens)
    vocab.append(tokens)
print(len(vocab))
for i in range(0,len(vocab)):
    for j in range(0,len(vocab[i])):
        vocab_size.add(vocab[i][j])
print(len(vocab_size))

## Model for word embeddings
Method to create model for training the word embeddings. Used sigmoid activation function and dropout with probab 0.2. The final layer has 1 neuron with relu activation function.

In [591]:
def get_model_nn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128, input_dim=300, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(tf.keras.layers.Dense(64, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(tf.keras.layers.Dense(24, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    print(model.summary())
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])
    return model

In [242]:
X1_train = X1.loc[:int(len(X1.index)*0.8)]
y1_train = X1_train['domain1_score']
X1_val = X1.loc[int(len(X1.index)*0.8)+1:int(len(X1.index)*0.9)]
y1_val = X1_val['domain1_score']
X1_test = X1.loc[int(len(X1.index)*0.9)+1:]
y1_test = X1_test['domain1_score']
y1_train = np.asarray(y1_train)
y1_val = np.asarray(y1_val)

In [241]:
print(X1.shape)
print(X1_train.shape)
print(X1_val.shape)
print(X1_test.shape)

(1783, 8)
(1427, 8)
(178, 8)
(178, 8)


Generating word embeddings for the essays

In [157]:
embedding_matrix = np.zeros((len(X1.index), 300))
print(embedding_matrix.shape)
for i in range(0, len(X1.index)):
    tokens = tokenize_words(X1.loc[i]['essay'])
    count = 0
    for token in tokens[1]:
        try:
            embedding_matrix[i] = np.add(embedding_matrix[i], glove_embeddings[token])
            count += 1
        except KeyError:
            pass
    embedding_matrix[i] = np.divide(embedding_matrix[i], count)

(1783, 300)


Splitting data into train, validation and test

In [243]:
embedding_matrix_train = embedding_matrix[:len(X1_train.index)]
embedding_matrix_val = embedding_matrix[len(X1_train.index):len(X1_train.index) + len(X1_val.index)]
embedding_matrix_test = embedding_matrix[len(X1_train.index) + len(X1_val.index):]

In [244]:
print(embedding_matrix_test.shape)
print(embedding_matrix_train.shape)
print(embedding_matrix_val.shape)

(178, 300)
(1427, 300)
(178, 300)


Loading the Neural Network Model.
This model is for the FeedForward Neural Network with Word Embeddings as the input.

In [599]:
embedding_model = get_model_nn()

Model: "sequential_111"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_244 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_245 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_246 (Dense)            (None, 24)                1560      
_________________________________________________________________
dense_247 (Dense)            (None, 1)                 25        
Total params: 48,369
Trainable params: 48,369
Non-trainable params: 0
_________________________________________________________________
None


Fitting the data i.e. training the model. Passing the train data for training and validation data for testing the model.

In [600]:
embedding_model.fit(embedding_matrix_train, y1_train, epochs=500,validation_data=(embedding_matrix_val,y1_val))

Train on 1427 samples, validate on 178 samples
Epoch 1/500
1427/1427 [==============================] - 2s 1ms/sample - loss: 38.3697 - accuracy: 0.0000e+00 - val_loss: 24.6450 - val_accuracy: 0.0000e+00
Epoch 2/500
1427/1427 [==============================] - 0s 97us/sample - loss: 21.7211 - accuracy: 0.0000e+00 - val_loss: 16.2883 - val_accuracy: 0.0000e+00
Epoch 3/500
1427/1427 [==============================] - 0s 99us/sample - loss: 14.8409 - accuracy: 0.0000e+00 - val_loss: 11.1954 - val_accuracy: 0.0000e+00
Epoch 4/500
1427/1427 [==============================] - 0s 122us/sample - loss: 10.2204 - accuracy: 0.0000e+00 - val_loss: 7.4841 - val_accuracy: 0.0000e+00
Epoch 5/500
1427/1427 [==============================] - 0s 98us/sample - loss: 6.5640 - accuracy: 0.0000e+00 - val_loss: 4.6224 - val_accuracy: 0.0000e+00
Epoch 6/500
1427/1427 [==============================] - 0s 97us/sample - loss: 3.9413 - accuracy: 0.0000e+00 - val_loss: 2.9621 - val_accuracy: 0.0000e+00
Epoch 7/50

Predict values of the test set and get the output. Apply the Quadratic Weighted Kappa metric for the final accuracy.

In [601]:
results = embedding_model.predict(embedding_matrix_test).flatten()
results = [int(r) for r in results]

In [602]:
percent = cohen_kappa_score(y1_test.values,results,weights='quadratic')
print(percent)

0.7173975169085194


Method to normalize the features data before using it for training

In [603]:
def normalize(data,mean,std):
    return ((data-mean)/std)

X1_norm = X1.copy(deep=True)
X1_norm['num_chars'] = X1_norm['num_chars'].apply(normalize,args=[X1.mean(axis=0)['num_chars'],X1.std(axis=0)['num_chars']])
X1_norm['num_words'] = X1_norm['num_words'].apply(normalize,args=[X1.mean(axis=0)['num_words'],X1.std(axis=0)['num_words']])
X1_norm['num_sents'] = X1_norm['num_sents'].apply(normalize,args=[X1.mean(axis=0)['num_sents'],X1.std(axis=0)['num_sents']])
X1_norm['avg_word_length'] = X1_norm['avg_word_length'].apply(normalize,args=[X1.mean(axis=0)['avg_word_length'],X1.std(axis=0)['avg_word_length']])
X1_norm['noun_count'] = X1_norm['noun_count'].apply(normalize,args=[X1.mean(axis=0)['noun_count'],X1.std(axis=0)['noun_count']])
X1_norm['adj_count'] = X1_norm['adj_count'].apply(normalize,args=[X1.mean(axis=0)['adj_count'],X1.std(axis=0)['adj_count']])
X1_norm['verb_count'] = X1_norm['verb_count'].apply(normalize,args=[X1.mean(axis=0)['verb_count'],X1.std(axis=0)['verb_count']])
X1_norm['adv_count'] = X1_norm['adv_count'].apply(normalize,args=[X1.mean(axis=0)['adv_count'],X1.std(axis=0)['adv_count']])
X1_norm['spelling_erros'] = X1_norm['spelling_erros'].apply(normalize,args=[X1.mean(axis=0)['spelling_erros'],X1.std(axis=0)['spelling_erros']])
X1_norm['avg_word_sent'] = X1_norm['avg_word_sent'].apply(normalize,args=[X1.mean(axis=0)['avg_word_sent'],X1.std(axis=0)['avg_word_sent']])
print(X1_norm.head())

   essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  num_chars  num_words  num_sents  avg_word_length  \
0              8  -0.221126  -0.277644  -0.746351        -0.007471   
1              9   0.358690   0.453147  -0.305963        -0.027029   
2              7  -0.701395  -0.714439  -0.966546        -0.131474   
3             10   1.706937   1.309936   0.464717         2.160992   
4              8   0.769102   0.822742   0.795009         0.333307   

   noun_count  adj_count  verb_count  adv_count  spelling_erros  avg_word_sent  
0   -0.532164  -0.708112   -0.20469

## Model for Features vector
Method to create model for training the features data of the essay. There are only 2 layers. The hidden layer as tanh as the activation function and the dropout is with probabiity 0.2. 

In [624]:
def get_features_model_nn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(10, input_dim=10, activation='tanh'))
#     model.add(Dropout(0.2))
    model.add(tf.keras.layers.Dense(4, activation='tanh'))
#     model.add(Dropout(0.2))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    print(model.summary())
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])
    return model

Creating feature vectors for training

In [605]:
features = X1_norm[['num_chars','num_words','num_sents','avg_word_length','noun_count','adj_count','verb_count','adv_count','spelling_erros','avg_word_sent','domain1_score']]
# print(features.head)
features_array = np.asarray(features)
features_array_train = features_array[:1427]
features_array_val = features_array[1427:1427+178]
features_array_test = features_array[1427+178:]
print(features_array_test.shape)
print(features_array_val.shape)
print(features_array_train.shape)

(178, 11)
(178, 11)
(1427, 11)


Loading the model and training it on the features data

In [626]:
features_model = get_features_model_nn()
features_model.fit(features_array_train[:,:-1], features_array_train[:,-1], epochs=500,validation_data=(features_array_val[:,:-1],features_array_val[:,-1]))

Model: "sequential_120"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_272 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_273 (Dense)            (None, 4)                 44        
_________________________________________________________________
dense_274 (Dense)            (None, 1)                 5         
Total params: 159
Trainable params: 159
Non-trainable params: 0
_________________________________________________________________
None
Train on 1427 samples, validate on 178 samples
Epoch 1/500
1427/1427 [==============================] - 1s 939us/sample - loss: 70.0459 - accuracy: 0.0000e+00 - val_loss: 62.6218 - val_accuracy: 0.0000e+00
Epoch 2/500
1427/1427 [==============================] - 0s 78us/sample - loss: 66.2628 - accuracy: 0.0000e+00 - val_loss: 58.6468 - val_accuracy: 0.0000e+00
Epoch 3/500
142

Predict values of the test set and get the output. Apply the Quadratic Weighted Kappa metric for the final accuracy.

In [627]:
features_results = features_model.predict(features_array_test[:,:-1]).flatten()
feature_results = [int(r) for r in features_results]

percent = cohen_kappa_score(features_array_test[:,-1],feature_results,weights='quadratic')
print(percent)

0.7812490745950428


## Model for word embeddings and feature vector combined
Method that gives the FeedForward neural network for the case where we combine the word embeddings and the feature vectors.

In [641]:
def get_combined_model_nn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128, input_dim=310, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(64, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(24, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    print(model.summary())
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])
    return model

Combining the word embeddings and the features vector

In [629]:
combined_matrix_train = np.concatenate((embedding_matrix_train, features_array_train[:,:-1]), axis=1)
combined_matrix_val = np.concatenate((embedding_matrix_val, features_array_val[:,:-1]), axis=1)
combined_matrix_test = np.concatenate((embedding_matrix_test, features_array_test[:,:-1]), axis=1)
print(combined_matrix_train.shape)
print(combined_matrix_val.shape)
print(combined_matrix_test.shape)

(1427, 310)
(178, 310)
(178, 310)


Fitting the data i.e. training the model. Passing the train data for training and validation data for testing the model.

In [652]:
combined_model = get_combined_model_nn()
combined_model.fit(combined_matrix_train, y1_train, epochs=500,validation_data=(combined_matrix_val,y1_val))

Model: "sequential_131"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_313 (Dense)            (None, 128)               39808     
_________________________________________________________________
dense_314 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_315 (Dense)            (None, 24)                1560      
_________________________________________________________________
dense_316 (Dense)            (None, 1)                 25        
Total params: 49,649
Trainable params: 49,649
Non-trainable params: 0
_________________________________________________________________
None
Train on 1427 samples, validate on 178 samples
Epoch 1/500
1427/1427 [==============================] - 1s 1ms/sample - loss: 47.0291 - accuracy: 0.0000e+00 - val_loss: 33.7578 - val_accuracy: 0.0000e+00
Epoch 2/500
1427/1427 [==============

Predict values of the test set and get the output. Apply the Quadratic Weighted Kappa metric for the final accuracy.

In [653]:
from sklearn.metrics import cohen_kappa_score
combined_results = combined_model.predict(combined_matrix_test).flatten()
combined_results = [int(r) for r in combined_results]
percent = cohen_kappa_score(y1_test,combined_results,weights='quadratic')
print(percent)

0.7886107472397761


## Creating the LSTM model for the word embeddings
Method that gives the LSTM model

In [696]:
def get_model_lstm():
    """Define the model."""
    
    model = Sequential()
    model.add(LSTM(128, input_shape=[1,300], activation = 'tanh',return_sequences=True))
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='relu'))

    model.summary()
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

    return model

Preprocessing data for passing it into the LSTM model

In [514]:
lstm_embed_train = []
for i in range(0,len(embedding_matrix_train)):
    lstm_embed_train.append(np.asarray([embedding_matrix_train[i]]))

lstm_embed_test = []
for i in range(0,len(embedding_matrix_test)):
    lstm_embed_test.append(np.asarray([embedding_matrix_test[i]]))

lstm_embed_val = []
for i in range(0,len(embedding_matrix_val)):
    lstm_embed_val.append(np.asarray([embedding_matrix_val[i]]))

lstm_embed_test = np.asarray(lstm_embed_test)
lstm_embed_train = np.asarray(lstm_embed_train)
lstm_embed_val = np.asarray(lstm_embed_val)
print(lstm_embed_train.shape)
print(lstm_embed_val.shape)
print(lstm_embed_test.shape)

(1427, 1, 300)
(178, 1, 300)
(178, 1, 300)


Fitting the data i.e. training the model. Passing the train data for training and validation data for testing the model.

In [699]:
lstm_model2 = get_model_lstm()
lstm_model2.fit(lstm_embed_train, y1_train, epochs=300,validation_data=(lstm_embed_val,y1_val))

Model: "sequential_152"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_103 (LSTM)              (None, 1, 128)            219648    
_________________________________________________________________
dropout_135 (Dropout)        (None, 1, 128)            0         
_________________________________________________________________
dense_359 (Dense)            (None, 1, 32)             4128      
_________________________________________________________________
dropout_136 (Dropout)        (None, 1, 32)             0         
_________________________________________________________________
dense_360 (Dense)            (None, 1, 1)              33        
Total params: 223,809
Trainable params: 223,809
Non-trainable params: 0
_________________________________________________________________
Train on 1427 samples, validate on 178 samples
Epoch 1/300
1427/1427 [==============================] - 3s 2ms/s

Predict values of the test set and get the output. Apply the Quadratic Weighted Kappa metric for the final accuracy.

In [700]:
from sklearn.metrics import cohen_kappa_score
lstm_result_2 = lstm_model2.predict(lstm_embed_test).flatten()
lstm_result_2 = [int(r) for r in lstm_result_2]

percent = cohen_kappa_score(y1_test,lstm_result_2,weights='quadratic')
print(percent)

0.6503508991231985
